In [1]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

In [2]:
# This is needed for autoformatting
# !pip install black[jupyter]
# !pip install --upgrade click==8.0.2
# from google.colab import drive
# drive.mount("/content/drive")
# !black /content/drive/MyDrive/Colab Notebooks/IuliaModel.ipynb

In [4]:
def logprob_dm(dm, params):
    # P(dm) log pdf
    cen, sig = params["dm_cen"], params["dm_sig"]
    N = scipy.stats.norm(cen)
    return N.logpdf(dm)


def logcdf_dm(dm, params):
    cen, sig = params["dm_cen"], params["dm_sig"]
    N = scipy.stats.norm(cen, sig)
    return N.logcdf(dm)


def logprob_dm_tracer(dm, params, extra):
    # P(dm|tracer)
    maglim1 = extra["mag_lim1"]
    maglim2 = extra["mag_lim2"]
    abs_mag = extra["abs_mag"]
    dm1 = maglim1 - abs_mag
    dm2 = maglim2 - abs_mag
    # normalized by the integral from dm1 to dm2
    return logprob_dm(dm, params) - (logcdf_dm(dm2, params) - logcdf_dm(dm1, params))


def logp_pm_tracer(mux, muy, dm, params):
    # P(proper motions conditional on distance modulus)

    mux0, muy0, mux_grad, muy_grad = (
        params["mux0"],
        params["muy0"],
        params["mux_grad"],
        params["muy_grad"],
    )
    N = len(mux)
    musig0, musig_grad = params["musig0"], params["musig_grad"]
    mu_vec = np.array([mux, muy]).T

    dm0 = 10
    mu_mean_pred = np.array(
        [mux0 + mux_grad * (dm - dm0), muy0 + muy_grad * (dm - dm0)]
    ).T
    sig2 = musig0 ** 2 * (1 + (dm - dm0) * musig_grad)
    covar_pred = np.array([[sig2, sig2 * 0], [sig2 * 0, sig2]])
    covar_pred = np.swapaxes(covar_pred, 0, 2)
    ret = 0
    for i in range(N):
        dmu = mu_vec[i] - mu_mean_pred[i]
        icovar = scipy.linalg.inv(covar_pred[i])
        curl = -0.5 * dmu.T @ icovar @ dmu + 0.5 * np.log(scipy.linalg.det(icovar))
        ret += curl
    return ret


def loglike(p, data, extra):
    # combined log-likelihood
    frac, mux0, muy0, mux_grad, muy_grad, musig0, musig_grad, dm_cen, dm_sig = p
    params = {
        "mux0": mux0,
        "muy0": muy0,
        "mux_grad": mux_grad,
        "muy_grad": muy_grad,
        "musig0": musig0,
        "musig_grad": musig_grad,
        "dm_cen": dm_cen,
        "dm_sig": dm_sig,
    }
    mux, muy, mags = data['mux'], data['muy'], data['mags']
    abs_mag1 = extra["abs_mag1"]
    abs_mag2 = extra["abs_mag2"]
    dm1 = mags - abs_mag1
    dm2 = mags - abs_mag2
    extra1 = extra.copy()
    extra2 = extra.copy()
    extra1["abs_mag"] = abs_mag1
    extra2["abs_mag"] = abs_mag2

    term1 = (
        np.log(frac)
        + logp_pm_tracer(mux, muy, dm1, params)
        + logprob_dm_tracer(dm1, params, extra1)
    )
    term2 = (
        np.log1p(-frac)
        + logp_pm_tracer(mux, muy, dm2, params)
        + logprob_dm_tracer(dm2, params, extra2)
    )

    ret = np.logaddexp(term1, term2)
    return ret.sum()

In [5]:
rng = np.random.default_rng(33)

In [6]:
def getdata(N, extra):
  mlim1 = extra['mag_lim1']
  mlim2 = extra['mag_lim2']
  abs_mag1 = extra['abs_mag1']
  abs_mag2 = extra['abs_mag2']
  mux = rng.normal(size=N)
  muy = rng.normal(size=N)
  mags = np.random.uniform(mlim1, mlim2, size=N)
  D =  {'mags':mags,'mux':mux, 'muy':muy}
  return D

extra = {'mag_lim1':10,'mag_lim2':15,'abs_mag1':0,'abs_mag2':3}
D=getdata(1000,extra)
p = [0.5, 0, 0, 0, 0, 1, 0, 10, 1]
print(loglike(p, D, extra))

-990298.0413592252


In [7]:
def get_covars(dm, params, extra):
    # get covariance matrices for each star
    cov_knots = extra["knots"]  # location of spline knots in dm
    n_knots = len(cov_knots)
    lsigma_x_vals = params["lsigma_x_vals"]  # log sigmax
    lsigma_y_vals = params["lsigma_y_vals"]  # log sigma y
    corr_coeff_vals = params["tan_rho_xy_vals"]  # tan(rho_xy * pi/2)
    SPLx = scipy.interpolate.CubicSpline(cov_knots, lsigma_x_vals)
    SPLy = scipy.interpolate.CubicSpline(cov_knots, lsigma_y_vals)
    SPLxy = scipy.interpolate.CubicSpline(cov_knots, corr_coeff_vals)
    sigma_x = np.exp(SPLx(dm))
    sigma_y = np.exp(SPLy(dm))
    sigma_xy = np.arctan(SPLxy(dm)) * 2 / np.pi * sigma_x * sigma_y
    mats = np.zeros((len(dm), 2, 2))
    mats[:, 0, 0] = sigma_x ** 2
    mats[:, 1, 1] = sigma_y ** 2
    mats[:, 0, 1] = sigma_xy
    mats[:, 1, 0] = sigma_xy
    return mats


print(
    get_covars(
        np.array([13, 14, 15]),
        extra={"knots": np.array([12, 14, 16])},
        params={
            "lsigma_x_vals": [1, 0.8, 0.6],
            "lsigma_y_vals": [1.2, 1, 0.8],
            "tan_rho_xy_vals": [-1, 0, 1],
        },
    )
)

[[[ 6.04964746 -2.18100726]
  [-2.18100726  9.0250135 ]]

 [[ 4.95303242  0.        ]
  [ 0.          7.3890561 ]]

 [[ 4.05519997  1.46197289]
  [ 1.46197289  6.04964746]]]
